# E6 - (연습) 작사가 인공지능 만들기

## 1. 데이터 다운로드

* `Song Lyrics` 데이터 다운로드 및 `Lyrics` 디렉토리 만들기

wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip  
unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  
#lyrics 폴더에 압축풀기

## 2. 데이터 읽어오기

* `glob` 모듈 : 파일 읽어오는 작업에 용이  
* `glob` 으로 모든 `txt` 파일을 읽어온 후, `raw_corpus` 리스트에 문장 단위로 저장

* 필요한 모듈 불러오기 및 데이터 읽어오기

In [1]:
import glob
import os
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## 3. 데이터 정제

* 문장 다듬기 (원하는 문장만 출력하기) : 불필요한 기호나 문장, 또는 공백 제거

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


* 텍스트 분류 모델에서 많이 보신 것처럼 텍스트 생성 모델에도 단어 사전을 만들게 됩니다. 그렇다면 문장을 일정한 기준으로 쪼개야겠죠? 그 과정을 토큰화(Tokenize) 라고 합니다.  
  
* 가장 심플한 방법은 띄어쓰기를 기준으로 나누는 방법이고, 우리도 그 방법을 사용할 겁니다. 하지만 약간의 문제가 있을 수 있죠. 몇 가지 문제 케이스를 살펴보죠.  
  
1. Hi, my name is John. *("Hi," "my", …, "john." 으로 분리됨) - 문장부호  
  
2. First, open the first chapter. *(First와 first를 다른 단어로 인식) - 대소문자  
   
3. He is a ten-year-old boy. *(ten-year-old를 한 단어로 인식) - 특수문자  
  
* "1." 을 막기 위해 문장 부호 양쪽에 공백을 추가 할 거고요, "2." 를 막기 위해 모든 문자들을 소문자로 변환할 겁니다. "3."을 막기 위해 특수문자들은 모두 제거하도록 하죠! 

* 이런 전처리를 위해 정규표현식(Regex)을 이용한 필터링

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


* 우리가 구축해야 할 데이터셋 모양  
  
언어 모델의 입력 문장 :  <start> 나는 밥을 먹었다 = 소스 문장(Source Sentence),  
언어 모델의 출력 문장 : 나는 밥을 먹었다 <end> = 타겟 문장(Target Sentence)

* 위에서 만든 정제 함수를 통해 만든 데이터셋에서 토큰화를 진행한 후 끝 단어 <end>를 없애면 소스 문장, 첫 단어 <start>를 없애면 타겟 문장  
* 이 정제 함수를 활용해서 아래와 같이 정제 데이터를 구축합니다!

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

* 데이터 준비 끝!!

## 4. 평가 데이터셋 분리

### (1) 데이터 토큰화 및 벡터화

* `tf.keras.preprocessing.text.Tokenizer` 패키지는 정제된 데이터를 토큰화  
* 단어 사전(vocabulary 또는 dictionary라고 칭함)을 만들어주며  
* 데이터를 숫자로 변환  
* 이 과정을 벡터화(vectorize) 라 하며, 숫자로 변환된 데이터를 텐서(tensor) 라고 칭합니다. 

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...    0    0    0]
 ...
 [   2   36    7 ...    0    0    0]
 [   2   13  440 ...    0    0    0]
 [   2   26   17 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f18641685d0>


In [6]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력해 보기
print(tensor[:3, :10])

[[   2   50    5   91  297   65   57    9  969 6042]
 [   2   17 2639  873    4    8   11 6043    6  329]
 [   2   36    7   37   15  164  282   28  299    4]]


* 텐서 데이터는 모두 정수로 구성  
* 이 숫자는 다름 아니라, tokenizer에 구축된 단어 사전의 인덱스

In [7]:
# 단어 사전이 어떻게 구축되었는지 아래와 같이 확인해 보기
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
print(len(tokenizer.index_word))

# 2번 인덱스가 `<start>` <- 모든 행이 2로 시작하는 이유 

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
27592


* 생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있게 하기  
* 이 과정에서도 텐서플로우가 제공하는 모듈을 사용  
* 텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분 : 정해진 입력 시퀀스 길이보다 문장이 짧은 경우 0으로 패딩(padding)을 채워넣은 것  
* 사전에는 없지만 0은 바로 패딩 문자 `<pad>`가 될 것임

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 `<end>`가 아니라 `<pad>`일 가능성 높음

src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    # tensor에서 `<start>` 잘라내서 타겟 문장 생성

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

* `corpus` 내의 첫번째 문장에 대해 생성된 소스와 타겟 문장을 확인해 보았음  
* 예상대로 소스는 2(`<start>`)에서 시작, 3(`<end>`)으로 끝난 후, 0(`<pad>`)로 채워져 있음  
* 하지만 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한칸 시프트한 형태를 띰

### (2) 데이터셋 객체 생성

* 그동안 `model.fit(x_train, y_train, …)` 형태로 `Numpy Array` 데이터셋을 생성하여 model에 제공하는 형태의 학습을 많이 진행  
* 텐서플로우를 활용할 경우 텐서로 생성된 데이터를 이용해 `tf.data.Dataset`객체를 생성하는 방법을 흔히 사용  
* `tf.data.Dataset`객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의기능을 제공하므로 꼭 사용법 알아 두기  
* 이미 데이터셋을 텐서 형태로 생성해 두었으므로, `tf.data.Dataset.from_tensor_slices()` 메소드를 이용해 `tf.data.Dataset`객체를 생성

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 4  # 배치사이즈 256일때, 메모리 어쩌고 에러가 떠서 그냥 확 줄여버림. 다른 해결법이 있었지만 아직 이해 불가!!
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1    
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((4, 346), (4, 346)), types: (tf.int32, tf.int32)>

### (3) 정리 요약 - "데이터 전처리"

* 데이터셋을 생성하기 위한 과정  
  
1. 정규표현식을 이용한 corpus 생성  
2. `tf.keras.preprocessing.text.Tokenizer`를 이용해 `corpus`를 텐서로 변환  
3. `tf.data.Dataset.from_tensor_slices()`를 이용해 `corpus` 텐서를 `tf.data.Dataset`객체로 변환  
  
* dataset을 얻음으로써 데이터 다듬기 과정은 끝  
* `tf.data.Dataset`에서 제공하는 `shuffle(), batch()` 등 다양한 데이터셋 관련 기능 이용  
  
* 이 모든 일련의 과정을 텐서플로우에서의 "데이터 전처리"라고 함

### (4) 훈련 데이터와 평가 데이터 분리

In [10]:
print(src_input.shape)

(175749, 346)


In [11]:
# 데이터 준비, 데이터 분리 관련 모듈들
from sklearn.model_selection import train_test_split

# 데이터 분리, 테스트데이터 사이즈 조절 및 랜덤성 결정
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input,
                                                    test_size = 0.2,
                                                    random_state = 0)
print('enc_train 개수: ', len(enc_train), ', enc_val 개수: ', len(enc_val))
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

enc_train 개수:  140599 , enc_val 개수:  35150
Source Train: (140599, 346)
Target Train: (140599, 346)


## 5. 인공지능 만들기

### (1) 모델 설계

* 만들 모델은 tf.keras.Model을 Subclassing하는 방식으로 만들 것  
* 만들 모델에는 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 16
hidden_size = 64
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

* 입력 텐서에는 단어 사전의 인덱스가 들어 있음    
* Embedding 레이어는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔 줌    
* 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용

* model 아직 제대로 build되지 않았음  
* `model.compile()`을 호출한 적도 없고, 아직 model의 입력 텐서가 무엇인지 제대로 지정해 주지도 않았기 때문  
* 이런 경우 아래와 같이 model에 데이터를 아주 조금 태워 보자!!  
* model의 input shape가 결정되면서 `model.build()`가 자동으로 호출됨

In [13]:
for src_input, tgt_input in dataset.take(1): break
model(src_input)

<tf.Tensor: shape=(4, 346, 7001), dtype=float32, numpy=
array([[[-1.01898862e-04, -2.46928375e-05,  1.25283666e-04, ...,
          6.28904236e-05,  1.14771747e-05,  3.32719683e-05],
        [-2.48694996e-04, -4.37231902e-05,  1.45203390e-04, ...,
          9.58687015e-05,  4.36506707e-06, -6.01506053e-06],
        [-2.00143942e-04, -6.15712270e-05,  9.43804480e-05, ...,
          9.34829950e-05, -1.35117016e-05, -1.10947312e-05],
        ...,
        [-1.39643077e-03,  1.30454777e-03,  1.06719614e-03, ...,
          2.07928111e-04, -6.25650806e-04, -4.54423745e-04],
        [-1.39643077e-03,  1.30454777e-03,  1.06719614e-03, ...,
          2.07928111e-04, -6.25650806e-04, -4.54423745e-04],
        [-1.39643077e-03,  1.30454777e-03,  1.06719614e-03, ...,
          2.07928111e-04, -6.25650806e-04, -4.54423745e-04]],

       [[-1.01898862e-04, -2.46928375e-05,  1.25283666e-04, ...,
          6.28904236e-05,  1.14771747e-05,  3.32719683e-05],
        [-1.95840650e-04, -1.03965736e-04,  2.0

* 모델의 최종 출력 텐서 shape를 유심히 보면 shape=(256, 20, 7001)  
  
* 7001은 Dense 레이어의 출력 차원수  
* 7001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링해야 하기 때문  
  
* 256은 이전 스텝에서 지정한 배치 사이즈  
* `dataset.take(1)`를 통해서 1개의 배치, 즉 256개의 문장 데이터를 가져온 것  
  
* 20은 `tf.keras.layers.LSTM(hidden_size, return_sequences=True)`로 호출한 LSTM 레이어에서 `return_sequences=True`이라고 지정한 부분  
* 즉, `LSTM`은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미  
* 만약 `return_sequences=False`였다면 `LSTM` 레이어는 1개의 벡터만 출력했을 것  
* 그런데 문제는, 우리의 모델은 입력 데이터의 시퀀스 길이가 얼마인지 모름  
* 모델을 만들면서 알려준 적도 없음  
* 그럼 20은 언제 알게된 것일까?? 데이터를 입력받고 나서...  
* 우리 데이터셋의 `max_len`이 20으로 맞춰져 있었음

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  112016    
_________________________________________________________________
lstm (LSTM)                  multiple                  20736     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33024     
_________________________________________________________________
dense (Dense)                multiple                  455065    
Total params: 620,841
Trainable params: 620,841
Non-trainable params: 0
_________________________________________________________________


* Output Shape를 정확하게 알려주지 않는 이유 : 우리의 모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없음  
* 하지만 모델의 파라미터 사이즈는 측정됨  
* 대략 22million 정도  
* 참고로 서두에 소개했던 GPT-2의 파라미터 사이즈는, 1.5billion  
* GPT-3의 파라미터 사이즈는 GPT-2의 100배

### (2) 모델 학습

* 학습엔 10분 정도 소요(GPU 환경 기준)  
* 혹시라도 학습에 지나치게 많은 시간이 소요된다면 `tf.test.is_gpu_available()` 소스를 실행해 텐서플로우가 GPU를 잘 사용하고 있는지 확인

In [15]:
# 아래 두 사이트 참고... 메모리 어쩌고 해결 방법이래서 해봤는데 효과는 모르겠다. 우선 여기까지만... 글고 뭔소린지 아직 이해 못함
# http://datacrew.tech/tensorflow-2%EC%97%90%EC%84%9C-gpu-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-1with-keras/
# https://cnpnote.tistory.com/entry/PYTHON-tensorflow%EC%97%90%EC%84%9C-%ED%98%84%EC%9E%AC-%EC%82%AC%EC%9A%A9-%EA%B0%80%EB%8A%A5%ED%95%9C-GPU%EB%A5%BC-%EC%96%BB%EB%8A%94-%EB%B0%A9%EB%B2%95

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [16]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)
# 미니배치가 각 배치마다 10에폭씩 실행되면서 업데이트하는 거라고 이해했는데, (그래서 한 에폭당 배치 개수만큼 업데이트??)

Epoch 1/10
43937/43937 [==============================] - 854s 19ms/step - loss: 0.1474
Epoch 2/10
43937/43937 [==============================] - 851s 19ms/step - loss: 0.1277
Epoch 3/10
43937/43937 [==============================] - 867s 20ms/step - loss: 0.1236
Epoch 4/10
43937/43937 [==============================] - 866s 20ms/step - loss: 0.1213
Epoch 5/10
43937/43937 [==============================] - 854s 19ms/step - loss: 0.1199
Epoch 6/10
43937/43937 [==============================] - 854s 19ms/step - loss: 0.1190
Epoch 7/10
43937/43937 [==============================] - 854s 19ms/step - loss: 0.1183
Epoch 8/10
43937/43937 [==============================] - 854s 19ms/step - loss: 0.1178
Epoch 9/10
43937/43937 [==============================] - 854s 19ms/step - loss: 0.1175
Epoch 10/10
43937/43937 [==============================] - 854s 19ms/step - loss: 0.1173


* Loss는 모델이 오답을 만들고 있는 정도라고 생각(그렇다고 Loss가 1일 때 99%를 맞추고 있다는 의미는 아님)  
* 오답률이 감소하고 있으니 학습이 잘 진행되고 있다 고 해석 가능!

### (3) 잘 만들어졌는지 평가하기

* 작문 모델을 평가하는 가장 확실한 방법은 작문을 시켜보고 직접 평가하는 것  
* 아래 `generate_text` 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하도록 함

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성
    while True:
        predict = model(test_tensor)
        # 입력받은 문장의 텐서를 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 모델이 예측한 마지막 단어가 새롭게 생성한 단어가 됨

        # 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줌
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 함
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장

* `generate_text()` 함수에서 `init_sentence`를 인자로 받고는 있음   
* 이렇게 받은 인자를 일단 텐서로 만들고 있음  
* 디폴트로는 `<start>` 단어 하나만 받음  
    
* `while`의 첫번째 루프에서 `test_tensor`에 `<start>` 하나만 들어갔다고 하고...    
* 모델이 출력으로 7001개의 단어 중 A를 골랐다고 할 때...  
* `while`의 두번째 루프에서 `test_tensor`에는 `<start> A`가 들어감  
* 그래서 모델이 그다음 B를 골랐다고 하면...  
* `while`의 세번째 루프에서 `test_tensor`에는 `<start> A B`가 들어감 .......

In [19]:
# 위 문장 생성함수 실행해 보기
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m a survivor <end> '

* 위 함수의 `init_sentence` 를 바꿔가며 이런저런 실험해 보기  
* 단, <start>를 빼먹지 말기!!

## 회고

이번 프로젝트는 앞의 연습을 그대로 따라하기만 해도 코드가 실행되어서 체감 어려움의 정도가 다른 것보다 낮았고 그 점이 너무 좋았다.  
하지만 모델 학습 시간이 굉장히 오래 걸려서 약간 마음을 가라앉히는 수행이 필요하였다...